![figure](lab6/politecnico_h-01.png)

# **Eletrónica Programável**
#### Licenciatura em Engenharia Eletrotécnica e de Computadores

## Trabalho Laboratorial 6 - Sistemas de Comunicação Série - UART e PS2

-----------------------


## Introdução ##
Neste trabalho vamos implementar e testar dois interfaces de comunicação: a) UART, que estabelece um sistema de comunicação série assíncrono; e b) PS2, que estabelece um sistema de comunicação série síncrono. Este documento não fornece detalhes de utilização do Vivado. Sempre que tiver dúvidas a este nível, consulte o enunciado do Lab1.


### Objetivos ###

Neste trabalho popomos a implementação três sistemas de comunicação, que são implementados recorrendo a máquinas de estados com datapath:
* Sistema de Comunicação UART bidireccional.
* Sistema de Comunicação PS2 para teclados.
* Sistema de Comunicação PS2 para ratos.


Nas instruções abaixo **{sources}** refere-se a `C:\Xilinx\EEC_EP\sources` e **{labs}** refere-se a `(C:\Xilinx\EEC_EP\labs)`


### Referencias ###

Pong P. Chu. 2008. FPGA Prototyping by VHDL Examples: Xilinx Spartan-3 Version. 


_______________________

## Passo 1 - UART bidireccional ##

### Passo 1.1 ### 

O diagrama de blocos de um UART bidireccional é representado na figura abaixo. É composto por um temporizador partilhado (*baud rate generator*), um transmissor, um recetor e duas memórias FIFO a servir de módulos de interface. Os módulos VHDL de cada um destes componentes são fornecidos nos ficheiros: **mod_m_counter.vhd, uart_tx.vhd, uart_rx.vhd e fifo.vhd**. Note o último módulo é o mesmo que foi utilizado no HandsOn 3. A descrição VHDL estrutural do diagrama de blocos é fornecida no ficheiro **uart.vhd**. 
 

<div>
<img src="lab6/fig1.png" width=700"/>
</div>
                                   

* Comece por descarregar os arquivos fonte fornecidos (Moodle) e salve-os em `{sources}/lab6`.


* Abra o Vivado a partir do icon no ambinete de trabalho ou: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Selecione **Create Project** para começar o wizard. Na caixa de diálogo *Create A New Vivado Project*, selecione **Next**.


* Clique no botão *Procurar* do campo de localização do projeto do novo formulário de projeto, navegue até `c:/Xilinx/EEC_EP/labs` e clique em **Select**. Digite *lab5* no campo de nome do projeto. Certifique-se de que a caixa *Create Project Subdirectory* esteja marcada. Clique em **Next**.


* Selecione a opção **RTL Project** no formulário *Project Type* e clique em **Next**. Certifique-se de que a caixa *Do not specify sources at this time* está marcada.


* No formulário **Default Part**, selecione a opção *Boards* e escolha a placa **Basys3**. Clique em **Next** e depois em **Finish** para criar o projeto Vivado.


* No painel *Sources*, selecione **Add Sources** e siga os passos necessários (*Add or create design sources*) para adicionar todos os ficheiros VHDL necessários (descritos acima).


* Abra o ficheiro **uart.vhd** no Vivado. Observe o código e verifique que ele descreve o diagrama de blocos acima apresentado. Note que o genérico DVSR (*baud rate divisor*) deve ter o valor de 326 caso o circuito seja implementado na placa (que possui um relógio de 100MHz) mas que pode ser mais pequeno (DVSR=2) para simulação.


* Adicione o ficheiro de estímulos de simulação (*testbench*) **uart_tb** ao projecto. Observe os processos que geram os estímulos de entrada. 


* Corra a simulação (com o genérico **DVSR=2**) e analise o funcionamento do sistema.


* Feche a vista de simulação e volte a colocar o enérico DVSR com o seu valor original (**DVSR=326**).



### Passo 1.2 ### 

Para verificar o funcionamento do UART na FPGA vamos usar o Hyperterminal de um PC e um botão da placa de desenvolvimento, tal como ilustra a figura abaixo. Neste circuito, o porto série (RS-232) da placa de desenvolvimento é ligado ao porto série de um PC. Os caracteres enviados a partir do PC devem ser recebidos pelo UART e armazenados na FIFO do recetor. Os dados armazenados são então alterados (por um circuito incrementador) e retransmitidos para o PC, através do UART. É usado um botão de pressão (com um circuito de debouncing) para controlar a leitura e escrita de dados na FIFO. 
 

<div>
<img src="lab6/fig2.png" width=500"/>
</div>
            
            
Por exemplo, quando escrevemos HAL no Hyperterminal e enviamos para a FPGA, a UART deve receber os caracteres e armazena-los na FIFO de recepção. Depois, se pressionarmos o botão de pressão uma vez, vamos ver a letra I a ser recebida no HyperTerminal (letra H +1). Pressionando mais duas vezes, recebemos os restantes caracteres B e M. 


* Adicione ao projeto os ficheiros que descrevem o módulo de teste (**uart_test.vhd**) e o circuito de debounce (**debounce.vhd**).


* Abra o ficheiro **uart_test.vhd** e observe que os dados recebidos podem ser visualizados num conjunto de 8 LEDs e que os sinais rx_full e rx_empty da FIFO de receção foram ligados a dois segmentos de um dos visores de 7 segmentos (visor 0). 


* Sintetize e implemente o módulo na FPGA. Lembre-se que deve primeiro definir este como sendo o módulo de topo (Set as Top Module), e criar um ficheiro de pinos de entrada/saída (com base nos sinais de entrada e saída desta entidade). 


* Ligue a Basys3 e programe-a. O cabo USB que usa para programar a placa servirá também de cabo de comunicação série, emulando uma ligação RS-232.


* Abra um Hyperterminal à sua escolha (sugiro o **TeraTerm**) para interagir com a placa de desenvolvimento, através do porto série. Este programa permite enviar caracteres no formato ASCII. Existem 128 caracteres definidos neste formato (alfabeto, pontuação e caracteres de controlo), que usa 7 bits para codificar cada caractere. Caso sinta necessidade, consulte a tabela de caracteres ASCII em http://www.asciitable.com/.


* Configure o Hyperterminal para **19200baud, 8 bits de dados, 1 stop bit, zero bits de paridade e nenhum controlo de fluxo (Flow Control)**. Se usar a aplicação TeraTerm para comunicar com o porto série, o procedimento a seguir é o seguinte.

  1.	Na janela **New Connection**, escolha comunicação série e selecione o último porto COM detectado. Faça ok. 

  2.	Vá a **Setup -> Serial Port**, e configure o porto de acordo com as especificações do UART fornecido (Baud rate: 19200; Data: 8 bits; Parity: none; Stop: 1bit, Flow Control: none). Faça ok. 

  3.	Vá a **Setup -> Terminal**, e escolha a opção **Local Echo**. Isto permitir-lhe-á observar os caracteres enviados no terminal. Faça Ok.


* Neste momento está pronto a comunicar com a placa de desenvolvimento. Observe o funcionamento da UART nos LEDs e também no HyperTerminal. Escreva os caracteres HAL no HyperTerminal e depois pressione o botão de pressão três vezes. Veja se observa o esperado nos LEDs e no HyperTerminal.


* Experimente com mais alguns caracteres e repare no que acontece quando tenta transmitir caracteres quando a memória do recetor está vazia (**rx_empty = '1'**) e o que acontece quando esta fica cheia (**rx_full = '1'**). Teste esta situação depois de um reset à memória e depois de a encher completamente. 



______

## Passo 2 - Interface PS2 a um teclado (PS2_RX) ##

### Passo 2.1 ### 

O circuito representado na figura abaixo representa um recetor PS2 para teclados, com extração do **make code** da tecla pressionada (vamos considerar que só é pressionada uma tecla de cada vez e que essa tecla é uma tecla regular, i.e., não estendida) e armazenamento numa FIFO de receção.


<div>
<img src="lab6/fig3.png" width="700"/>
</div>


Os dados PS2 são recebidos no módulo **ps2_rx.vhd**, e enviados para uma FIFO (a mesma usada no passo 1 deste trabalho, logo já adicionada ao projeto), sendo todo este procedimento controlado por uma máquina de estados. No estado **wait_brk** a FSM monitoriza constantemente os pacotes recebidos. Quando o pacote **F0** (break code) é detectado, ela passa para o estado **get_code** e espera pelo próximo pacote, que já se sabe ser o make code da tecla pressionada. Depois, máquina assere o sinal **code_done_tick = '1'** durante um ciclo de relógio e retorna ao estado inicial (**wait_brk**). 


* Adicione ao projeto os ficheiros **ps2_rx.vhd** e **kb_code.vhd** (que instancia os outrs módulos e define a FSM). Abra este último ficheiro e observe o código que descreve a FSM.


* Adicione agora ao projeto o ficheiro **kb_test.vhd**, que descreve o diagramade blocos fornecido abaixo. Este circuito permite receber o make code da tecla pressionada, converter para código ASCII e enviar para o Hyperterminal do PC via UART. Abra os ficheiros e verifique como é implementado este sistema.


<div>
<img src="lab6/fig4.png" width="700"/>
</div>


* Implemente este circuito na Basys 3. Lembre-se que é necessário definir o módulo **kb_test.vhd** como sendo o módulo de topo (**Set as Top**) e atualizar o ficheiro de pinos de entrada/saída. 


* Programe a placa, ligue um teclado na entrada USB e verifique o funcionamento do teclado PS2 através do HyperTerminal do PC. 


* Tente responder às seguintes questões:

  1. Por que motivo é recebido o caractere * no HyperTerminal quando pressiona a tecla SHIFT? Onde (no código) é especificado tal comportamento?

  2. Porque motivo só aparece no HyperTerminal o código de uma tecla quando ela é libertada, e não quando é pressionada?


______________

## Passo 3 - Interface PS2 a um rato (PS2_RX + PS2_TX) ##

### Passo 3.1 ### 

Combinando os subsistemas de transmissão e recepção, podemos formar um sistema de comunicação bidirecional, tal como é necessário para usar um rato PS2. O diagrama de blocos deste sistema é representado na figura pelo bloco azul. Os sinais **tx_idle** e **rx_en** servem para coordenar as operações de transmissão e recepção. O código com a descrição VHDL estrutural deste circuito é fornecido no ficheiro **ps2_rxtx.vhd**. O módulo ps2_tx é fornecido no ficheiro **ps2_tx.vhd**.

<div>
<img src="lab6/fig5.png" width="500"/>
</div>


O interface completo do rato, fornecido no ficheiro **mouse.vhd**, cria uma nova camada protocolar no sistema de comunicação. O circuito tem como saídas: **xm e ym**, que correspondem aos movimentos nos eixos do X e do Y; **btnm**, que indica o estado do circuito em 3 bits; e o sinal **m_done_tick**, que fica asserido quando os dados estão disponíveis. 

A camada protocolar é implementada como uma FSMD, com sete estados. Os estados **init1, init2 e init3** são executados após a reinicialização do sistema, para colocar o rato no stream mode. Estados **pack1, pack2 e pack3** são então usados para receber os 3 pacotes de dados. No estado final, **done**, é asserido o sinal m_done_tick. A máquina circula então entre estes últimos 4 estados.

Para testar o circuito na placa, é fornecido um circuito que usa o rato para controlar os LEDs da placa de desenvolvimento. Deve acender apenas 1 LED, que acompanha o movimento do rato no eixo do X. Pressionando o botão direito ou esquerdo do rato, deve acender o LED da esquerda ou o LED da direita. 


* Adicione ao projecto o ficheiro com a descrição VHDL deste circuito (**mouse_led.vhd**).


* Implemente o circuito na Basys3. Lembre-se que deve primeiro defini-lo como sendo o módulo de topo e depois criar um ficheiro de pinos de entrada/saída. 


* Programe a placa e verifique o seu funcionamento, usando um rato ligado à estrada USB-A da Basys3.


* Analise o código do módulo de topo e responda às questões que se seguem.
 
 1. Quando desloca o rato no eixo do yy os LEDs não respondem. Porquê?
 
 2. Quando desloca o rato no eixo do yy, os LEDs acendem de forma lenta. Se é suposto o rato incrementar os seus contadores em 4 unidades a cada milímetro, porque motivo acendem de forma tão lenta?
 
 3. No ficheiro mouse.vhd há uma máquina de estados com 7 estados. Porque são necessários 7 estados e qual a função de cada um?



_________

## Desafio ##

Crie um ficheiro de estímulos que permita simular o funcionamento do circuito **kb_code.vhd**. Simule a chegada de dados relativos a um evento de tecla **A** pressionada (1C F0 1C). Note que terá de gerar os sinais ps2d e ps2c com as temporizações adequadas e com os valores corretos: 1) a frequência do relógio clk deve ser muito superior à frequência do relógio ps2c por causa do filtro; 2) a vertente descendente do ps2c deve estar a meio dos dados em ps2d; 3) o sinal em ps2d deve iniciar com um start bit, depois os dados (LSB primeiro), parity e stop bit; 4) as linhas ps2c e ps2d estão normalmente a ‘1’.

Corra a simulação e verifique se o circuito funciona de acordo com o diagrama de fluxo apresentado nos slides da aula TP. 


